In [ ]:
import math

import torch
from torch import Tensor
import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [ ]:
type_1 = torch.zeros(128, 1, 1000)  # 10个样本，每个样本 1×100
# type_2 = torch.ones(10, 1, 12)
# labels = F.one_hot(torch.tensor([0,0,0,0,0,0,0,0,0,0]))

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)
                             * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
class VATN(nn.Module):
    def __init__(self, ):
        super(VATN, self).__init__()
        self.conv = nn.Conv1d(in_channels=1, out_channels=256, kernel_size=50, stride=25)
        self.pos_encoder = PositionalEncoding(256)
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=256, nhead=8, dim_feedforward=1024, batch_first=True),
            num_layers=8)
        self.max_pool = nn.AdaptiveMaxPool1d(4)
        self.avg_pool = nn.AdaptiveAvgPool1d(4)
        self.flatten = nn.Flatten()
        self.fc_1 = nn.Linear(312, 64)
        self.fc_2 = nn.Linear(64, 4)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.conv(x)
        x = x.transpose(1, 2)
        x = self.pos_encoder(x)
        x = self.encoder(x)
        x_1 = self.max_pool(x)
        x_2 = self.avg_pool(x) 
        x = torch.cat((x_1, x_2), dim=2)
        x = self.flatten(x)
        x = self.fc_1(x)
        x = F.relu(x)
        x = self.fc_2(x)
        x = self.softmax(x)
        return x

In [ ]:
model = VATN()
out = model(type_1)
out.shape

In [ ]:
from utils.model import VATN

In [ ]:
model = VATN()
out = model(type_1)
out.shape

In [ ]:
def train(model, num_epochs=1):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            # data = data.to(device)
            # target = target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target.to(torch.float32))
            loss.backward()
            optimizer.step()
            
            if batch_idx % 3 == 0:
                print(output)
                print(target.shape)
                print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                        f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
                
    print('Finished Training')